In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ecommerce-purchase-history-from-electronics-store/kz.csv


In [2]:
import sqlite3
import csv

data = pd.read_csv('/kaggle/input/ecommerce-purchase-history-from-electronics-store/kz.csv')

data.head(5)

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id
0,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
1,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
2,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
3,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
4,2020-04-24 19:16:21 UTC,2294584263154074236,2273948316817424439,2.268105e+18,NaN,karcher,217.57,1.515916e+18


In [3]:
conn = sqlite3.connect("test.db")
data.to_sql('kz', conn)

In [4]:
#example of SQL script 
cur = conn.cursor()
cur.execute("SELECT * FROM kz")
print(cur.fetchone())

(0, '2020-04-24 11:50:39 UTC', 2294359932054536986, 1515966223509089906, 2.268105426648171e+18, 'electronics.tablet', 'samsung', 162.01, 1.515915625441994e+18)


In [5]:
df = pd.read_sql("SELECT order_id FROM kz", conn, index_col=None)
df.head(5)

,order_id
0,2294359932054536986
1,2294359932054536986
2,2294444024058086220
3,2294444024058086220
4,2294584263154074236


In [6]:
#Calculating the feature quantity for the entire table
'''
df = pd.read_sql("SELECT 
				   count(*) AS total_count
				  , count(distinct product_id) as product_count 
				  , count(distinct order_id) as order_count
				  , count(distinct category_id) as category_count
				  , count(distinct user_id) as user_count
				  , sum(price) as sum
				  , AVG(price) as avg
				  , max(price) as max
				  , min(price) as min
				 from kz")
				 
'''

df = pd.read_sql("SELECT  count(*) AS total_count, count(distinct product_id) as product_count , count(distinct order_id) as order_count, count(distinct category_id) as category_count, count(distinct user_id) as user_count, sum(price) as sum, AVG(price) as avg, max(price) as max, min(price) as min from kz", conn, index_col=None)

df

,total_count,product_count,order_count,category_count,user_count,sum,avg,max,min
0,2633521,25113,1435266,900,98262,3.392464e+08,154.093165,50925.9,0.0


In [7]:
#Calculating the feature quantity for users
'''
df = pd.read_sql("SELECT  user_id,
					count(*) AS total_count
				  , count(distinct product_id) as product_count 
				  , count(distinct order_id) as order_count
				  , count(distinct category_id) as category_count				  
				  , sum(price) as sum
				  , AVG(price) as avg
				  , max(price) as max
				  , min(price) as min
				 from kz
				 GROUP BY user_id 
				 order by sum desc
")
				 

'''
df = pd.read_sql("SELECT  user_id, count(*) AS total_count, count(distinct product_id) as product_count, count(distinct order_id) as order_count, count(distinct category_id) as category_count, sum(price) as sum, AVG(price) as avg, max(price) as max, min(price) as min from kz GROUP BY user_id order by sum desc", conn, index_col=None)
df

,user_id,total_count,product_count,order_count,category_count,sum,avg,max,min
0,NaN,2069352,17917,1056723,657,2.217473e+08,135.426653,50925.90,0.00
1,1.515916e+18,635,460,402,134,1.217013e+05,191.655591,1157.38,0.23
2,1.515916e+18,632,521,364,154,1.118839e+05,177.031472,1736.09,0.00
3,1.515916e+18,584,493,328,140,1.088258e+05,186.345599,4745.35,0.67
4,1.515916e+18,548,470,342,132,9.496708e+04,173.297591,3009.24,0.23
...,...,...,...,...,...,...,...,...,...
98258,1.515916e+18,1,1,1,1,0.000000e+00,0.000000,0.00,0.00
98259,1.515916e+18,1,1,1,1,0.000000e+00,0.000000,0.00,0.00
98260,1.515916e+18,1,1,1,1,0.000000e+00,0.000000,0.00,0.00
98261,1.515916e+18,1,1,1,1,0.000000e+00,0.000000,0.00,0.00


In [8]:
#Calculating the feature quantity for product
'''
df = pd.read_sql("SELECT  product_id,
   count(*) AS total_count
  ,count (distinct product_id)  as product_count
  , count(distinct user_id) as user_count 
  , count(distinct order_id) as order_count
  , count(distinct category_id) as category_count
  , sum(price) as sum
  , AVG(price) as avg
  , max(price) as max
  , min(price) as min
    from kz
    GROUP BY product_id
    order by order_count desc
    limit 100
)


'''

df = pd.read_sql("SELECT  product_id,count(*) AS total_count  ,count (distinct product_id)  as product_count, count(distinct user_id) as user_count   , count(distinct order_id) as order_count  , count(distinct category_id) as category_count  , sum(price) as sum  , AVG(price) as avg  , max(price) as max  , min(price) as min from kz GROUP BY product_id order by order_count desc limit 100", conn, index_col=None)
df




,product_id,total_count,product_count,user_count,order_count,category_count,sum,avg,max,min
0,1515966223523303302,44491,1,0,44491,0,NaN,NaN,NaN,NaN
1,1515966223523303301,41076,1,0,41076,0,NaN,NaN,NaN,NaN
2,1515966223523303308,38786,1,0,38786,0,NaN,NaN,NaN,NaN
3,1515966223523303310,38472,1,0,38472,0,NaN,NaN,NaN,NaN
4,1515966223523303314,36530,1,0,36530,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
95,1515966223510204922,2624,1,399,2624,1,9052.80,3.45,3.45,3.45
96,2273948312304353950,2618,1,182,2617,1,362933.34,138.63,138.63,138.63
97,1515966223509089963,2604,1,471,2602,1,271180.56,104.14,104.14,104.14
98,1515966223509088671,2587,1,1254,2587,1,2394708.29,925.67,925.67,925.67


If you check the above result, you can see that price sum appears as a null value in the case of top 1.
As a result, it can be assumed that the order was placed, but the actual sales were not connected.
I do not know the exact reason, but I know that there are cases where a lot of orders are placed for reasons such as refund or sales cancellation, but sales do not occur.
Therefore, it is judged that it will be meaningful to extract the data of the top 100 sales by generating the order based on the sales.

In [9]:
#Calculating top 100 for product sales
'''
df = pd.read_sql("SELECT  product_id,
   count(*) AS total_count
  , count(distinct user_id) as user_count 
  , count(distinct order_id) as order_count
  , sum(price) as sum
  , AVG(price) as avg
  , max(price) as max
  , min(price) as min
    from kz
    GROUP BY product_id
    order by sum desc
    limit 100
)
'''
df = pd.read_sql("SELECT  product_id,   count(*) AS total_count  , count(distinct user_id) as user_count   , count(distinct order_id) as order_count  , sum(price) as sum  , AVG(price) as avg  , max(price) as max  , min(price) as min    from kz    GROUP BY product_id    order by sum desc    limit 100", conn, index_col=None)
df


,product_id,total_count,user_count,order_count,sum,avg,max,min
0,1515966223509088567,4336,1921,4336,3712613.28,856.23,856.23,856.23
1,1515966223509088671,2587,1254,2587,2394708.29,925.67,925.67,925.67
2,1515966223509089284,1967,623,1967,2390396.75,1215.25,1215.25,1215.25
3,1515966223509088532,6470,1953,6466,1946823.00,300.90,300.90,300.90
4,1515966223509088628,2244,846,2242,1921896.24,856.46,856.46,856.46
...,...,...,...,...,...,...,...,...
95,1515966223509106938,1246,212,1246,519133.44,416.64,416.64,416.64
96,2273948254817223175,2010,281,2010,511766.10,254.61,254.61,254.61
97,1515966223509088619,628,255,628,508780.48,810.16,810.16,810.16
98,2273948272861118569,1454,146,1453,504828.80,347.20,347.20,347.20


## 2. Identify monthly characteristics
After the data extraction exercise for basic analysis is over, now it is time to figure out the monthly characteristics in earnest.

The easiest way to access is monthly analysis, let's check monthly category sales

### monthly category sales
